In [1]:
from PIL import Image
import numpy as np
import os

In [2]:
def loadImagesToArray(path:str):
    '''
    Loads all .jpg and .png files from the specified directory.\n
    Each image will be converted into an array of size (height x width x channels).\n
    The return numpy array is of dimensions (numberOfImages x height x width x channels).\n
    '''
    imagesArray = []

    counter = 0
    for file in os.scandir(path):
        filepath = os.fsdecode(file)
        if(filepath.endswith(".jpg") or filepath.endswith(".png")):
                imgArray = np.array(Image.open(filepath))
                imagesArray.append(imgArray)
                counter += 1                  
    return np.array(imagesArray)

def loadTrainingDataAndLabels(path:str, subdirectories):
    '''
    Loads the training data as numpy arrays and creates the corresponding labels.\n
    For this to work, the images should be under the folder <path> in separate subdirectories, one for each class.\n
    The labels will be inferred from the names of the subdirectories. \n

    Returns the training data as a numpy array with the dimensions (number_of_images x height x width x channels).\n
    Returns the labels as a numpy array with the dimensions (number_of_images).
    '''

    training_data = []
    labels = []

    for directory in subdirectories:
        images_array = loadImagesToArray(os.path.join(path, directory))
        training_data.extend(images_array)

        labels.extend(np.full(len(images_array), directory))

    training_data_array = np.array(training_data)
    print("Shape of training_data: ", training_data_array.shape)
    labels_array = np.array(labels)
    print("Shape of labels: ", labels_array.shape)
    
    return training_data_array, labels_array

In [3]:
training_data, labels = loadTrainingDataAndLabels("task_2/training_patches/", ["background", "ponds", "pools", "solar", "trampoline"])

Shape of training_data:  (3316, 256, 256, 3)
Shape of labels:  (3316,)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(training_data, labels, test_size=0.33, random_state=1, stratify=labels)

In [5]:
y_train = y_train.reshape(len(y_train),1)
y_train.shape

(2221, 1)

In [6]:
y_val = y_val.reshape(len(y_val),1)
y_val.shape

(1095, 1)

In [7]:
import pandas as pd
dftrain = pd.DataFrame(data=y_train)
y_dumtrain = pd.get_dummies(dftrain)
y_dumtrain.to_numpy()

dfval = pd.DataFrame(data=y_val)
y_dumval = pd.get_dummies(dfval)
y_dumval.to_numpy()

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]], dtype=uint8)

In [8]:
print(y_dumtrain.shape)
print(y_dumval.shape)

(2221, 5)
(1095, 5)


In [9]:
from tensorflow.keras.utils import to_categorical

def prepare_data(X_train, y_train, X_val, y_val):
    
    # integers to floats
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    
    # normalize image data (to obtain values between 0 and 1)
    X_train = X_train / 255.0
    X_val = X_val / 255.0
    
    # one-hot encoding for our targets 
    
    return X_train, y_train, X_val, y_val

X_train, y_train, X_val, y_val = prepare_data(X_train, y_dumtrain, X_val, y_dumval) 

KeyError: 1

In [11]:
import matplotlib.pyplot as plt

import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras

In [12]:
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D

model = keras.models.Sequential()
model.add(InputLayer(input_shape=(256,256,3)))
model.add(Conv2D(filters=10, kernel_size=(3,3), strides=1, padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(20, activation="relu"))
model.add(Dense(20, activation="relu"))
model.add(Dense(5, activation="softmax"))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 10)      280       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 10)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 163840)            0         
                                                                 
 dense (Dense)               (None, 20)                3276820   
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
 dense_2 (Dense)             (None, 5)                 105       
                                                        

In [14]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
history = model.fit(X_train, 
                    y_train, 
                    epochs=15,
                    batch_size=100,
                    validation_split=0.1,
                   )

Epoch 1/15


2022-07-08 14:43:35.838985: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


20/20 [==============================] - 7s 293ms/step - loss: 0.5510 - accuracy: 0.9174 - val_loss: 0.5349 - val_accuracy: 0.9238
Epoch 2/15
20/20 [==============================] - 6s 279ms/step - loss: 0.3249 - accuracy: 0.9394 - val_loss: 0.4571 - val_accuracy: 0.9238
Epoch 3/15
20/20 [==============================] - 6s 273ms/step - loss: 0.2796 - accuracy: 0.9394 - val_loss: 0.4282 - val_accuracy: 0.9238
Epoch 4/15
20/20 [==============================] - 6s 299ms/step - loss: 0.2556 - accuracy: 0.9394 - val_loss: 0.4197 - val_accuracy: 0.9238
Epoch 5/15
20/20 [==============================] - 7s 342ms/step - loss: 0.2182 - accuracy: 0.9424 - val_loss: 0.3804 - val_accuracy: 0.9283
Epoch 6/15
20/20 [==============================] - 6s 303ms/step - loss: 0.1768 - accuracy: 0.9565 - val_loss: 0.3349 - val_accuracy: 0.9372
Epoch 7/15
20/20 [==============================] - 6s 304ms/step - loss: 0.1415 - accuracy: 0.9620 - val_loss: 0.3125 - val_accuracy: 0.9372
Epoch 8/15
20/20 

In [43]:

pred = model.predict(X_val)


35/35 [==============================] - 1s 31ms/step


In [27]:
y_val.iloc[10]

0_background    1
0_ponds         0
0_pools         0
0_solar         0
0_trampoline    0
Name: 10, dtype: uint8

In [44]:
for idx1,i in enumerate(pred):
    print(i)
    for idx2,j in enumerate(i):
        print(j)
        if (j>0.8):
            pred[idx1][idx2] = 1
        else:
            pred[idx1][idx2]=0
        

[9.9942350e-01 2.4541505e-04 6.2042469e-05 1.6712533e-04 1.0183046e-04]
0.9994235
0.00024541505
6.204247e-05
0.00016712533
0.00010183046
[9.9836165e-01 1.7714321e-04 2.1115791e-04 2.1488630e-04 1.0351387e-03]
0.99836165
0.00017714321
0.00021115791
0.0002148863
0.0010351387
[9.9962795e-01 1.5483705e-04 2.0796306e-05 9.7271099e-05 9.9136167e-05]
0.99962795
0.00015483705
2.0796306e-05
9.72711e-05
9.913617e-05
[9.9731678e-01 3.2904846e-04 1.3482130e-04 3.1941597e-04 1.8999012e-03]
0.9973168
0.00032904846
0.0001348213
0.00031941597
0.0018999012
[9.9549013e-01 7.2152889e-04 2.3145418e-04 7.5858628e-04 2.7983158e-03]
0.99549013
0.0007215289
0.00023145418
0.0007585863
0.0027983158
[9.9876368e-01 4.5782255e-04 2.7765005e-04 3.2163769e-04 1.7932334e-04]
0.9987637
0.00045782255
0.00027765005
0.0003216377
0.00017932334
[9.9990582e-01 1.9001433e-05 1.6782815e-05 2.0812306e-05 3.7659884e-05]
0.9999058
1.9001433e-05
1.6782815e-05
2.0812306e-05
3.7659884e-05
[9.9980134e-01 1.8977860e-05 7.0033975e-06 

In [45]:
pred

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val,pred)

0.9525114155251142

In [51]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val.values.argmax(axis=1), pred.argmax(axis=1))

array([[1027,    0,    0,    0,    0],
       [   3,    0,    0,    0,    0],
       [   5,    0,    0,    0,    2],
       [  10,    0,    0,    0,    2],
       [  30,    0,    0,    0,   16]])

In [54]:
from sklearn.metrics import f1_score
f1_score(y_val,pred,average="macro")

0.29361881137850504